In [1]:
import cv2
import typing
import numpy as np
import os


from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer, get_wer
from mltu.transformers import ImageResizer

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list: typing.Union[str, list], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    def predict(self, image: np.ndarray):
        image = ImageResizer.resize_maintaining_aspect_ratio(image, *self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.char_list)[0]

        return text

if __name__ == "__main__":
    import requests
    
    # URL of the image you want to download
    image_url = 'https://static-cse.canva.com/blob/1145059/35gravityhandwritten.jpg'
    response = requests.get(image_url)
    image_data = response.content
    from mltu.configs import BaseModelConfigs

    configs = BaseModelConfigs.load("./configs.yaml")

    model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    name = ""
    
    if response.status_code == 200:
    # Get the content of the response (the image data)
        image_data = response.content

    # Specify the file name where you want to save the image
        file_name = 'downloaded_image.jpg'
        name = file_name

    # Open the file in binary write mode and save the image data
        with open(file_name, 'wb') as f:
            f.write(image_data)
    image = cv2.imread(name)

    prediction_text = model.predict(image)
    print("Prediction: ", prediction_text)
    
    try:
        os.remove(name)
        print(f"{name} deleted.")
    except OSError as e:
        print(f"Error deleting {name}: {e}")






Prediction:  Gavity ftandwuitten
downloaded_image.jpg deleted.


In [38]:
import onnx
import tensorflow as tf
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load('./model.onnx')

# Convert ONNX to TensorFlow
tf_rep = prepare(onnx_model)
tf.saved_model.save(tf_rep, '')

ModuleNotFoundError: No module named 'tensorflow_probability'